Copyright (c) Microsoft Corporation.
Licensed under the MIT License.

# Abstractive Summarization using BertSumAbs on CNN/DailyMails Dataset

## Summary

This notebook demonstrates how to fine tune BERT for abstractive text summarization. Utility functions and classes in the NLP Best Practices repo are used to facilitate data preprocessing, model training, model scoring, result postprocessing, and model evaluation.

### Abstractive Summarization
Abstractive summarization is the task of taking an input text and summarizing its content in a shorter output text. In contrast to extractive summarization, abstractive summarization doesn't take sentences directly from the input text, instead, rephrases the input text.

### BertSumAbs

BertSumAbs refers to an BERT-based abstractive summarization algorithm  in [Text Summarization with Pretrained Encoders](https://arxiv.org/abs/1908.08345) with [published examples](https://github.com/nlpyang/PreSumm). It uses the pretrained BERT model as encoder and finetune both encoder and decoder on a specific labeled summarization dataset like [CNN/DM dataset](https://github.com/harvardnlp/sent-summary). 

The figure below shows the comparison of architecture of the original BERT model (left) and BERTSUM (right), which BertSumAbs is built upon. For BERTSUM, a input document is split into sentences, and [CLS] and [SEP] tokens are inserted before and after each sentence. This resulting sequence is followed by the summation of three kinds of embeddings for each token before feeding into the transformer layers. The positional embedding used in BertSumAbs enables input length of more than 512, which is the  maximum input length for BERT model. 

It should be noted that the architecture only shows the encoder part. For decoder, BertSumAbs also uses a transformer with multiple layers and random initialization. As pretrained weights are used in the encoder, there is a mismatch in encoder and decoder which may result in unstable finetuning. Therefore, in fine tuning, BertSumAbs uses seperate optimizers for encoder and decoder, each uses its own scheduling. In text generation, techniques like trigram blocking and beam search can be used to improve model accuracy.
<img src="https://nlpbp.blob.core.windows.net/images/BertForSummarization.PNG">


## Before you start
Set QUICK_RUN = True to run the notebook on a small subset of data and a smaller number of steps. If QUICK_RUN = False, the notebook takes about 5 hours to run on a VM with 4 16GB NVIDIA V100 GPUs.

To run rouge evaluation, please refer to the section of compute_rouge_perl in [summarization_evaluation.ipynb](./summarization_evaluation.ipynb).

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
QUICK_RUN = False

In [3]:
import os
import shutil
import sys
from tempfile import TemporaryDirectory
import torch

nlp_path = os.path.abspath("../../")
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)

from utils_nlp.models.transformers.abstractive_summarization_bertsum import BertSumAbs, BertSumAbsProcessor

from utils_nlp.dataset.cnndm import CNNDMSummarizationDataset
from utils_nlp.eval.evaluate_summarization import get_rouge

import pandas as pd
import scrapbook as sb

/home/daden/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/daden/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/daden/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/daden/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: Futu

## Data Preprocessing

The dataset we used for this notebook is CNN/DM dataset which contains the documents and accompanying questions from the news articles of CNN and Daily mail. The highlights in each article are used as summary. The dataset consits of ~289K training examples, ~11K valiation examples and ~11K test examples. The length of the news articles is 781 tokens on average and the summaries are of 3.75 sentences and 56 tokens on average.

The significant part of data preprocessing only involve splitting the input document into sentences.

In [4]:
# the data path used to save the downloaded data file
DATA_PATH =  TemporaryDirectory().name
# The number of lines at the head of data file used for preprocessing. -1 means all the lines.
TOP_N = 100
if not QUICK_RUN:
    TOP_N = -1

In [5]:
train_dataset, test_dataset = CNNDMSummarizationDataset(
            top_n=TOP_N, local_cache_path=DATA_PATH, prepare_extractive=False
        )

100%|██████████| 489k/489k [00:07<00:00, 63.8kKB/s] 


In [6]:
len(train_dataset.get_source()[0])

45

In [7]:
len(test_dataset.get_target()[0])

4

## Model Finetuning

In [8]:
# notebook parameters
# the cache path
CACHE_PATH = TemporaryDirectory().name

# model parameters
MODEL_NAME = "bert-base-uncased"
MAX_POS = 768
MAX_SOURCE_SEQ_LENGTH = 640
MAX_TARGET_SEQ_LENGTH = 140

# mixed precision setting. To enable mixed precision training, follow instructions in SETUP.md. 
FP16 = False
if FP16:
    FP16_OPT_LEVEL="O2"
    
# fine-tuning parameters
# batch size, unit is the number of tokens
BATCH_SIZE_PER_GPU = 3


# GPU used for training
NUM_GPUS = torch.cuda.device_count()

# Learning rate
LEARNING_RATE_BERT=2e-3
LEARNING_RATE_DEC=0.2


# How often the statistics reports show up in training, unit is step.
REPORT_EVERY=10
SAVE_EVERY=500

# total number of steps for training
MAX_STEPS=1e2
   
if QUICK_RUN:
    MAX_STEPS=5e4

WARMUP_STEPS_BERT=20000
WARMUP_STEPS_DEC=10000   


In [9]:
# processor which contains the colloate function to load the preprocessed data
processor = BertSumAbsProcessor(cache_dir=CACHE_PATH, max_src_len=MAX_SOURCE_SEQ_LENGTH, max_tgt_len=MAX_TARGET_SEQ_LENGTH)
# summarizer
summarizer = BertSumAbs(
    processor, cache_dir=CACHE_PATH, max_pos_length=MAX_POS
)

In [10]:
BATCH_SIZE_PER_GPU*NUM_GPUS

12

In [11]:

summarizer.fit(
        train_dataset,
        num_gpus=NUM_GPUS,
        batch_size=BATCH_SIZE_PER_GPU*NUM_GPUS,
        max_steps=MAX_STEPS,
        learning_rate_bert=LEARNING_RATE_BERT,
        learning_rate_dec=LEARNING_RATE_DEC,
        warmup_steps_bert=WARMUP_STEPS_BERT,
        warmup_steps_dec=WARMUP_STEPS_DEC,
        save_every=SAVE_EVERY,
        report_every=REPORT_EVERY,
        fp16=FP16,
)


device is cuda


/dadendev/anaconda3/envs/cm3/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


timestamp: 11/03/2020 04:02:16, average loss: 14.456905, time duration: 37.727674,
                            number of examples in current reporting: 240, step 10
                            out of total 100
timestamp: 11/03/2020 04:02:35, average loss: 14.018371, time duration: 19.310867,
                            number of examples in current reporting: 240, step 20
                            out of total 100
timestamp: 11/03/2020 04:02:56, average loss: 13.277190, time duration: 20.774043,
                            number of examples in current reporting: 240, step 30
                            out of total 100
timestamp: 11/03/2020 04:03:16, average loss: 11.843365, time duration: 19.581301,
                            number of examples in current reporting: 240, step 40
                            out of total 100
timestamp: 11/03/2020 04:03:37, average loss: 10.200771, time duration: 20.827822,
                            number of examples in current reporting: 240, ste

In [12]:
summarizer.save_model(MAX_STEPS, os.path.join(CACHE_PATH, "./bertsumabs.pt"))

saving through pytorch to /tmp/tmpsmbejg9y/./bertsumabs.pt


## Model Evaluation

In [13]:
# del summarizer
# checkpoint = torch.load("./abstemp/bert-base-uncased_step_1000.pt")
# summarizer.model.load_checkpoint(checkpoint['model'])

In [14]:
# clear cache
import gc; gc.collect()
torch.cuda.empty_cache()

In [15]:
TOP_N=32
shortened_dataset= test_dataset.shorten(top_n=TOP_N)
src = shortened_dataset.get_source()
reference_summaries = [" ".join(t).rstrip("\n") for t in shortened_dataset.get_target()]
generated_summaries = summarizer.predict(
    shortened_dataset, batch_size=32, num_gpus=NUM_GPUS
)
assert len(generated_summaries) == len(reference_summaries)


Generating summary:   0%|          | 0/4 [00:00<?, ?it/s]

dataset length is 128


Generating summary: 100%|██████████| 4/4 [02:49<00:00, 42.50s/it]


In [16]:
src[0]

['marseille , france ( cnn ) the french prosecutor leading an investigation into the crash of germanwings flight 9525 insisted wednesday that he was not aware of any video footage from on board the plane .',
 'marseille prosecutor brice robin told cnn that " so far no videos were used in the crash investigation . "',
 'he added , " a person who has such a video needs to immediately give it to the investigators . "',
 "robin 's comments follow claims by two magazines , german daily bild and french paris match , of a cell phone video showing the harrowing final seconds from on board germanwings flight 9525 as it crashed into the french alps .",
 'all 150 on board were killed .',
 'paris match and bild reported that the video was recovered from a phone at the wreckage site .',
 'the two publications described the supposed video , but did not post it on their websites .',
 'the publications said that they watched the video , which was found by a source close to the investigation . "',
 'on

In [17]:
generated_summaries[0]

'[MASK] [MASK] [MASK]      . . .   the the    . . , , . . to              .'

In [18]:
reference_summaries[0]

' marseille prosecutor says " so far no videos were used in the crash investigation " despite media reports .    journalists at bild and paris match are " very confident " the video clip is real , an editor says .    andreas lubitz had informed his lufthansa training school of an episode of severe depression , airline says .  '

In [19]:
RESULT_DIR = TemporaryDirectory().name
rouge_score = get_rouge(generated_summaries, reference_summaries, RESULT_DIR)
print(rouge_score)

2020-03-11 04:09:55,582 [MainThread  ] [INFO ]  Writing summaries.
INFO - Writing summaries.
2020-03-11 04:09:55,583 [MainThread  ] [INFO ]  Processing summaries. Saving system files to /tmp/tmp5yxefp9m/tmpvwkaulln/system and model files to /tmp/tmp5yxefp9m/tmpvwkaulln/model.
INFO - Processing summaries. Saving system files to /tmp/tmp5yxefp9m/tmpvwkaulln/system and model files to /tmp/tmp5yxefp9m/tmpvwkaulln/model.
2020-03-11 04:09:55,584 [MainThread  ] [INFO ]  Processing files in /tmp/tmp5yxefp9m/rouge-tmp-2020-03-11-04-09-55/candidate/.
INFO - Processing files in /tmp/tmp5yxefp9m/rouge-tmp-2020-03-11-04-09-55/candidate/.
2020-03-11 04:09:55,610 [MainThread  ] [INFO ]  Saved processed files to /tmp/tmp5yxefp9m/tmpvwkaulln/system.
INFO - Saved processed files to /tmp/tmp5yxefp9m/tmpvwkaulln/system.
2020-03-11 04:09:55,611 [MainThread  ] [INFO ]  Processing files in /tmp/tmp5yxefp9m/rouge-tmp-2020-03-11-04-09-55/reference/.
INFO - Processing files in /tmp/tmp5yxefp9m/rouge-tmp-2020-03

128
128
---------------------------------------------
1 ROUGE-1 Average_R: 0.04577 (95%-conf.int. 0.03970 - 0.05184)
1 ROUGE-1 Average_P: 0.31823 (95%-conf.int. 0.27404 - 0.37294)
1 ROUGE-1 Average_F: 0.07693 (95%-conf.int. 0.06722 - 0.08672)
---------------------------------------------
1 ROUGE-2 Average_R: 0.00109 (95%-conf.int. 0.00022 - 0.00208)
1 ROUGE-2 Average_P: 0.00556 (95%-conf.int. 0.00112 - 0.01004)
1 ROUGE-2 Average_F: 0.00182 (95%-conf.int. 0.00036 - 0.00345)
---------------------------------------------
1 ROUGE-L Average_R: 0.04152 (95%-conf.int. 0.03618 - 0.04722)
1 ROUGE-L Average_P: 0.29869 (95%-conf.int. 0.25191 - 0.35434)
1 ROUGE-L Average_F: 0.07000 (95%-conf.int. 0.06129 - 0.07937)

{'rouge_1_recall': 0.04577, 'rouge_1_recall_cb': 0.0397, 'rouge_1_recall_ce': 0.05184, 'rouge_1_precision': 0.31823, 'rouge_1_precision_cb': 0.27404, 'rouge_1_precision_ce': 0.37294, 'rouge_1_f_score': 0.07693, 'rouge_1_f_score_cb': 0.06722, 'rouge_1_f_score_ce': 0.08672, 'rouge_2_reca

In [20]:
# for testing
sb.glue("rouge_2_f_score", rouge_score['rouge_2_f_score'])

## Clean up temporary folders

In [21]:
if os.path.exists(DATA_PATH):
    shutil.rmtree(DATA_PATH, ignore_errors=True)
if os.path.exists(CACHE_PATH):
    shutil.rmtree(CACHE_PATH, ignore_errors=True)
if os.path.exists(RESULT_DIR):
    shutil.rmtree(RESULT_DIR, ignore_errors=True)